In [1]:
import os, glob, re
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import json
import xlsxwriter

In [2]:
# TODO add calibration adjustments

In [ ]:
def convert_to_dataframe(data_tables, injection_number=None, injection_time=None):
    '''
    Combines list of extracted data tables into two pandas dataframes (one for FID and one for TCD)
    data_tables = list of parsed REPORT.txt data, cf. extract_tables_from_file
    injection_number = number of injection [int]
    injection_time = datetime of injection
    '''
    # Skip header, units, and separator
    FID_data = data_tables[0][4:]
    TCD_data = data_tables[1][4:]
    #data_lines = table_lines[4:]

    FID_df = PFR_parse_data(FID_data)
    TCD_df = PFR_parse_data(TCD_data)

    for df in (FID_df, TCD_df):
        df['Injection #'] = injection_number
        df['Injection Time'] = injection_time

    return FID_df, TCD_df


def PFR_parse_data(data):
    '''
    Parses REPORT.txt data from lines of text to pandas df
    '''
    parsed_data = []
    for line in data:

        line = line.strip()

        match = re.match(
            r"(\d+)\s+([\d.]+)\s+([A-Z ]+?)\s+([\d.]+)\s+([\d.eE+-]+)\s+([\d.eE+-]+)\s+([\d.]+)", 
            line
        )
        if match:
            parsed_data.append(match.groups())
        else:
            print(f"❌ Failed to parse: {repr(line)}")


    column_names = ['Peak', 'RetTime [min]', 'Type', 'Width', 'Area', 'Height', 'Percent']
    df = pd.DataFrame(parsed_data, columns=column_names)

    for col in ['Peak', 'RetTime [min]', 'Width', 'Area', 'Height', 'Percent']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return df


def extract_tables_from_file(filepath, start_marker_FID, start_marker_TCD, end_marker):
    '''
    Extracts lines of FID and TCD data from REPORT.txt as a list of text
    filepath = file location of injection folder containing REPORT.txt
    start_marker_FID = common string before lines of FID data are shown in REPORT.txt
    start_marker_TCD = common string before lines of TCD data are shown in REPORT.txt
    end_marker = common string after FID/TCD data has been displayed in REPORT.txt
    '''
    with open(filepath, 'r', encoding='utf-16', errors='ignore') as f:
        lines = f.readlines()

    FID_table_lines = []
    TCD_table_lines = []
    inside_table = False
    FID = False
    TCD = False

    for line in lines:
        if start_marker_FID in line:
            inside_table = True
            FID = True
            continue  # skip the start line itself
        elif start_marker_TCD in line:
            inside_table = True
            TCD = True
            continue
        elif 'Injection Date' in line:
            injection_time = datetime.strptime(re.search(r'Injection Date\s+: ([\d/]+\s+\d{1,2}:\d{2}:\d{2}\s+[AP]M)', line).group(1),"%m/%d/%Y %I:%M:%S %p")
            continue

        # TODO simplify 
        if inside_table:
            if line.strip().startswith(end_marker) and FID and not TCD:  # In FID table
                FID = False
                inside_table = False  # end of table
            elif line.strip().startswith(end_marker) and not FID and TCD: # In TCD table
                TCD = False
                inside_table = False  # end of table
                if FID:
                    print('FID is still inside_table')
                break
            elif FID and not TCD:
                FID_table_lines.append(line.strip())
            else:
                TCD_table_lines.append(line.strip())

    return (FID_table_lines, TCD_table_lines), injection_time


In [ ]:
# Configuration for input data, output file, and data to be collected
dir_path = r"C:\...\experiment_folder"
file_name = "experiment_data.xlsx"
excel_file_path = os.path.join(dir_path, file_name)
fid_sheet_name = 'All_FID_data'
tcd_sheet_name = 'All_TCD_data'
extracted_sheet_name = 'Extracted_area_data'
desired_peaks = ['Ethylene','Ethylene Oxide','Ethyl Chloride','Ethane','Oxygen','Carbon Dioxide']



# Extract REPORT.txt files with GC data
all_sequences_FID = {}
all_sequences_TCD = {}

for seq_folder in os.listdir(dir_path):
    seq_path = os.path.join(dir_path, seq_folder)
    if not os.path.isdir(seq_path):
        continue

    print(f"Processing sequence: {seq_folder}")
    all_FID = []
    all_TCD = []

    # iterate over injections
    for inj_folder in os.listdir(seq_path):
        inj_path = os.path.join(seq_path, inj_folder)
        if not os.path.isdir(inj_path):
            continue

        # find REPORT.txt
        text_files = glob.glob(os.path.join(inj_path, 'REPORT.txt'))
        if not text_files:
            print(f"WARNING: {inj_folder} does not have a REPORT file.")
            continue

        file_path = text_files[0]

        try:
            # extract raw tables
            tables, injection_time = extract_tables_from_file(
                file_path,
                start_marker_FID="Signal 1: FID1 A,",
                start_marker_TCD="Signal 2: TCD2 B,",
                end_marker="Totals :"
            )

            # parse injection number and sequence number from folder name (adapt if needed)
            m = re.match(r'001F(\d{2})(\d{2})\.D', inj_folder)
            if m:
                injection_number, sequence_number = m.groups()
                if int(sequence_number) != 1:
                    print(f"WARNING: Sequence number {sequence_number} found, skipping {inj_folder}")
                    continue
            else:
                injection_number = inj_folder  # fallback if no regex match

            # convert to DataFrames
            FID_df, TCD_df = convert_to_dataframe(tables, injection_number, injection_time)

            all_FID.append(FID_df)
            all_TCD.append(TCD_df)

        except Exception as e:
            print(f"Error processing {inj_folder}: {e}")
            continue

    # combine all injections for this sequence
    if all_FID:
        all_sequences_FID[seq_folder] = pd.concat(all_FID, ignore_index=True)
    if all_TCD:
        all_sequences_TCD[seq_folder] = pd.concat(all_TCD, ignore_index=True)



Processing sequence: 02kPa O2
Processing sequence: 04 kPa O2
Processing sequence: 06 kPa O2
Processing sequence: 10kPa O2
Processing sequence: 15 kPa O2
❌ Failed to parse: '1 8.41e-3 BB S  0.0228 1.51764e-1 1.10854e-1  0.00089'


In [ ]:
# Label data based on retention time data from configuration files
with open('../FID_peaks.json','r') as f:
    FID_RetTime = json.load(f)

with open('../TCD_peaks.json','r') as f:
    TCD_RetTime = json.load(f)

all_sequences_FID_peaks = {}
all_sequences_TCD_peaks = {}


# TODO combine FID/TCD filters somehow so it only runs once
for seq_name, FID_master in all_sequences_FID.items():
    FID_peak_dfs = {}
    for peak_name, (low,high) in FID_RetTime.items():
        mask = (low <= FID_master['RetTime [min]']) & (FID_master['RetTime [min]'] <= high)
        peaks = FID_master[mask]
        FID_peak_dfs[peak_name] = peaks
    all_sequences_FID_peaks[seq_name] = FID_peak_dfs

for seq_name, TCD_master in all_sequences_TCD.items():
    TCD_peak_dfs = {}
    for peak_name, (low,high) in TCD_RetTime.items():
        mask = (low <= TCD_master['RetTime [min]']) & (TCD_master['RetTime [min]'] <= high)
        peaks = TCD_master[mask]
        TCD_peak_dfs[peak_name] = peaks
    all_sequences_TCD_peaks[seq_name] = TCD_peak_dfs



In [ ]:
# TODO Make excel writers write to path instead of this folder
# TODO Combine FID and TCD into one file
# TODO use Bronkhorts logger to automatically label, or even reformat for instant use

# Organize data from injection data df and compile to excel
with pd.ExcelWriter(excel_file_path, engine="xlsxwriter") as writer: # change FID path to path + name

    # Extract area data
    extracted_area_frames = []

    for seq_name in set(all_sequences_FID_peaks.keys()) | set(all_sequences_TCD_peaks.keys()):
        seq_area = []

        # FID peaks
        if seq_name in all_sequences_FID_peaks:
            for peak_name, df in all_sequences_FID_peaks[seq_name].items():
                if peak_name in desired_peaks and 'Injection Time' in df.columns and 'Area' in df.columns:
                    temp = df[["Injection Time", "Area"]].copy()
                    temp = temp.groupby('Injection Time').sum(numeric_only=True).reset_index()
                    temp.rename(columns={'Area': peak_name}, inplace=True)
                    seq_area.append(temp)

        # TCD peaks
        if seq_name in all_sequences_TCD_peaks:
            for peak_name, df in all_sequences_TCD_peaks[seq_name].items():
                if peak_name in desired_peaks and 'Injection Time' in df.columns and 'Area' in df.columns:
                    temp = df[["Injection Time", "Area"]].copy()
                    temp = temp.groupby('Injection Time').sum(numeric_only=True).reset_index()
                    temp.rename(columns={'Area': peak_name}, inplace=True)
                    seq_area.append(temp)

        # Merge all peaks side-by-side
        if seq_area:
            merged = seq_area[0]
            for nxt in seq_area[1:]:
                merged = pd.merge(merged, nxt, on="Injection Time", how="outer")
            merged["Sequence"] = seq_name
            extracted_area_frames.append(merged)

    # Combine
    print(extracted_area_frames)
    extracted_area_df = pd.concat(extracted_area_frames, ignore_index=True)
    # Ensure Injection Time + Sequence come first, then peaks in desired order
    ordered_cols = ["Sequence", "Injection Time"] + [p for p in desired_peaks if p in extracted_area_df.columns]
    extracted_area_df = extracted_area_df[ordered_cols]
    extracted_area_df.to_excel(writer, sheet_name=extracted_sheet_name, index=False)



    # Save FID results
    fid_frames = []
    for seq_name, peaks_dict in all_sequences_FID_peaks.items():
        for peak_name, df in peaks_dict.items():
            temp = df.copy()
            temp["Peak"] = peak_name
            temp["Sequence"] = seq_name
            fid_frames.append(temp)
    all_fid_df = pd.concat(fid_frames, ignore_index=True)
    all_fid_df.to_excel(writer, sheet_name=fid_sheet_name, index=False)

    # Save TCD results
    tcd_frames = []
    for seq_name, peaks_dict in all_sequences_TCD_peaks.items():
        for peak_name, df in peaks_dict.items():
            temp = df.copy()
            temp["Peak"] = peak_name
            tcd_frames.append(temp)
    all_tcd_data = pd.concat(tcd_frames, ignore_index=True)
    all_tcd_data.to_excel(writer, sheet_name=tcd_sheet_name, index=False)

[        Injection Time      Ethylene  Ethylene Oxide  Ethyl Chloride  \
0  2025-11-18 12:09:02  26436.166540             NaN       67.659710   
1  2025-11-18 12:23:52   8683.204100       384.47278        2.608920   
2  2025-11-18 12:38:23   7786.543950       590.54590        2.067240   
3  2025-11-18 12:53:20   7347.910640       669.11938        1.796690   
4  2025-11-18 13:08:00   2633.779540       393.92902        2.030960   
5  2025-11-18 13:22:59   6885.516600       718.25159        1.517260   
6  2025-11-18 13:38:01   6803.828130       734.77289        1.532100   
7  2025-11-18 13:53:04   6731.523440       736.67328        1.483390   
8  2025-11-18 14:08:01   6671.290530       743.50946        1.472910   
9  2025-11-18 14:23:08   6665.261230       755.32477        1.397770   
10 2025-11-18 14:38:04   6612.316410       761.67737        1.397520   
11 2025-11-18 14:53:08   6535.656740       770.51440        1.393000   
12 2025-11-18 15:08:11   6604.963870       750.15704        1.3